The LLM model is supported by the Azure OPENAI service. One can also use OPENAI API to run this code.

In [1]:
import os
import sys
parent_dir = os.path.abspath('../..')
sys.path.append(parent_dir)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from eval_utils import compare_MLE, data_profiling, cate_info, col_check,  transform_label
from openai import AzureOpenAI
from sklearn.preprocessing import LabelBinarizer
import os
import json
import pickle
from model_glm import MultiAgentGAN

In [2]:
data = pd.read_csv('../../../data/adult_imputed.csv')

In [2]:
data100 = pd.read_csv("../sample/Adult/data100.csv")
data200 = pd.read_csv("../sample/Adult/data200.csv")
data400 = pd.read_csv("../sample/Adult/data400.csv")
data800 = pd.read_csv("../sample/Adult/data800.csv")
data_test = pd.read_csv("../sample/Adult/data_test.csv")

In [7]:
len(data)

32561

In [8]:
data.shape

(32561, 15)

In [9]:
data.drop(columns=['Unnamed: 0'], inplace=True)
data = data.rename(columns = {'label': 'Income'})

In [3]:
x_cols = ['age', 'workclass', 'education', 'education-num',
          'marital-status', 'occupation', 'relationship', 'race', 'sex',
          'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
y_col = 'Income'
cate_var = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'Income']
bool_var = []
cols = x_cols + [y_col]

In [4]:
api_version = "2023-05-15"

api_key1 = 777# Your api_key
azure_endpoint1 = "xxx.com/" # Your end point

api_key2 = 777 # Your api_key
azure_endpoint2 = "xxx.com/" # Your end point

gen_client = AzureOpenAI(
            api_key = api_key1,
            api_version = api_version,
            azure_endpoint = azure_endpoint1
        )

opt_client = AzureOpenAI(
    api_key = api_key2,
    api_version = api_version, 
    azure_endpoint = azure_endpoint2
)

gen_model_nm = 'generator4'

opt_model_nm = 'yaobin_gpt4'

params = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'objective': 'binary:logistic',  # logistic regression for binary classification, output probability
    'eval_metric': 'logloss'  # evaluation metric for binary classification
}
data_desc = "The dataset include people's social economic factors and demographics with the label that indicates whether their income is higher than 50k."
num_var = list((set(cols) - set(cate_var)) - set(bool_var))
num_var

['capital-loss', 'education-num', 'age', 'hours-per-week', 'capital-gain']

In [12]:
data = data[cols].copy()

In [22]:
data_test = data.sample(200, random_state=42)
data_curated = data.drop(data_test.index)
data100 = data_curated.sample(n = 100, replace=False, random_state=42)
data200 = data_curated.sample(n = 200, replace=False, random_state=42)
data400 = data_curated.sample(n = 400, replace=False, random_state=42)
data800 = data_curated.sample(n = 800, replace=False, random_state=42)
print(set(data100.index).intersection(set(data_test.index)))
data100.reset_index(inplace=True, drop=True)
data200.reset_index(inplace=True, drop=True)
data400.reset_index(inplace=True, drop=True)
data800.reset_index(inplace=True, drop=True)

data_test.to_csv("../sample/Adult/data_test.csv", index=False)
data100.to_csv("../sample/Adult/data100.csv", index=False)
data200.to_csv("../sample/Adult/data200.csv", index=False)
data400.to_csv("../sample/Adult/data400.csv", index=False)
data800.to_csv("../sample/Adult/data800.csv", index=False)

set()


In [7]:
res = {}  
model_dict = {}
num_samples = [100, 200, 400, 800]
sampled_datasets = [data100, data200, data400, data800]
epochs = [5, 4, 3, 2]
batch_size = 50
seed_lst = list(range(5))
df_test = data_test.reset_index(drop=True).copy()

In [ ]:
for i in range(1, 4):
    num = num_samples[i]
    sampled_data = sampled_datasets[i]
    epoch = epochs[i]
    res[str(num)] = {}
    df_comb = pd.concat([sampled_data, df_test])
    df_comb.reset_index(inplace=True, drop = True)
    metadata = data_profiling(df_comb, cate_var, bool_var, cols)
    dummy_cols = list(pd.get_dummies(df_comb[x_cols]).columns)
    cate_desc = cate_info(df_comb, cate_var)
    df_test_dummy = pd.get_dummies(df_test[x_cols])
    df_test_dummy = col_check(df_test_dummy, dummy_cols)
    X_test = df_test_dummy.to_numpy()
    y_test = df_test[y_col].to_numpy()
    log_file = f'log/adult_{num}.txt'
    magan = MultiAgentGAN(gen_client, opt_client, gen_model_nm, opt_model_nm, params, sampled_data, cols, y_col, num_var, metadata, cate_desc, data_desc, log_file, opt_temperature=1, real_samples_num=5)
    magan._run(batch_size, epoch)
    magan.run_with_fixed_discriminator(0, epoch, batch_size, 1)
    model_dict[str(num)] = magan
    
    for j in range(len(seed_lst)):
        seed = seed_lst[j]
        res[str(num)][str(seed)] = {}
        mallm_syn_df = magan.gen_without_optimization(num_folds=1)
        mallm_syn_df.to_csv(f'gen/adult/{num}/df_{seed}.csv', index=False)